<a href="https://colab.research.google.com/github/Nish-011-100/Predicting-Semiconductor-Properties-Using-ML/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pymatgen


In [2]:
import importlib.metadata
print(importlib.metadata.version("pymatgen"))
!pip uninstall pymatgen -y
!pip install --upgrade pymatgen
!pip install pymatgen matgl

2025.2.18
Found existing installation: pymatgen 2025.2.18
Uninstalling pymatgen-2025.2.18:
  Successfully uninstalled pymatgen-2025.2.18
  Using cached pymatgen-2025.2.18-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached pymatgen-2025.2.18-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.1 MB)


In [6]:
!pip install kaggle pymatgen matgl dgl -f https://data.dgl.ai/wheels/repo.html


Looking in links: https://data.dgl.ai/wheels/repo.html


In [9]:

# !pip install kaggle pymatgen matgl

# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import networkx as nx
# from pymatgen.core import Structure
# #from matgl.ext.pymatgen import StructureGraphConverter
# #from matgl.graph.converters import CrystalGraphConverter
# from sklearn.model_selection import train_test_split, GridSearchCV
# from xgboost import XGBRegressor
# from sklearn.svm import SVR
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from sklearn.preprocessing import StandardScaler
# import os

# # Set up Kaggle API key (Make sure you have uploaded kaggle.json to Colab or set up locally)
# os.environ['KAGGLE_CONFIG_DIR'] = "./"

# # Download Semiconductor dataset from Kaggle
# !kaggle datasets download -d sudalairajkumar/semiconductor-properties -p ./ --unzip

# df = pd.read_csv("./semiconductor_properties.csv")

# def clean_data(df):
#     df = df.dropna()
#     df = df[df['band_gap'] > 0]  # Ensure only semiconductors are considered
#     return df

# df = clean_data(df)

# print("✅ Semiconductor dataset loaded successfully from Kaggle.")



403 - Forbidden - Permission 'datasets.get' was denied


FileNotFoundError: [Errno 2] No such file or directory: './semiconductor_properties.csv'

In [ ]:
!kaggle datasets list -s semiconductor
from google.colab import files
files.upload()  # Upload kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
import os
import pandas as pd

# Redownload dataset
!kaggle datasets download -d sudalairajkumar/semiconductor-properties -p ./ --unzip

# Check available files
files = os.listdir("./")
print("📂 Files in directory:", files)

# Load CSV file dynamically
csv_files = [f for f in files if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError("❌ No CSV file found. Check the dataset or download process.")

df = pd.read_csv(csv_files[0])  # Automatically selects first CSV file
print("✅ Dataset loaded:", df.head())




Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in ./. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [12]:
# from pymatgen.ext.matproj import MPRester
# import pandas as pd

# API_KEY = "YOUR_NEW_API_KEY"  # Replace with your new API key

# with MPRester(API_KEY) as m:
#     # Query semiconductor materials using the latest MPRester
#     data = m.summary.search(
#         band_gap=(0, None),  # Get materials with bandgap > 0
#         fields=["material_id", "formula_pretty", "band_gap", "density",
#                 "energy_per_atom", "formation_energy_per_atom", "efermi"]
#     )

# df = pd.DataFrame([d.model_dump() for d in data])  # Use `model_dump()` for Pydantic models
# df.to_csv("semiconductor_data.csv", index=False)

# print("✅ Semiconductor dataset successfully retrieved from Materials Project.")




AttributeError: '_MPResterLegacy' object has no attribute 'summary'

In [13]:
from sklearn.preprocessing import LabelEncoder

# Remove non-numerical columns
df = df.drop(columns=["pretty_formula"])

# Label encode categorical variables (if any)
encoder = LabelEncoder()
df["space_group"] = encoder.fit_transform(df["space_group"])


NameError: name 'df' is not defined

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = df.drop(columns=["band_gap"])  # Features
y = df["band_gap"]  # Target variable

X_scaled = scaler.fit_transform(X)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, r2_score

model = SVR(kernel="rbf")
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}, R² Score: {r2}")


In [ ]:
import requests
import pandas as pd

url = "http://aflowlib.duke.edu/API/aflux/?species=Si,Ge,Ga,As"
response = requests.get(url).json()

df = pd.DataFrame(response)
df.to_csv("aflow_semiconductors.csv", index=False)


In [ ]:
# Fill missing values with mean or drop them
df = df.fillna(df.mean())


In [ ]:
df = pd.get_dummies(df, columns=["crystal_structure"], drop_first=True)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = df.drop(columns=["band_gap"])
y = df["band_gap"]

X_scaled = scaler.fit_transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score

model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}, R² Score: {r2}")


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    "kernel": ["rbf", "poly", "sigmoid"],
    "C": [0.1, 1, 10, 100],
    "gamma": ["scale", "auto", 0.01, 0.1, 1]
}

# Perform Grid Search
svr = SVR()
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best MAE:", -grid_search.best_score_)


In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

# Define parameter grid
param_grid = {
    "n_estimators": [50, 100, 200, 500],
    "learning_rate": [0.01, 0.1, 0.2, 0.3],
    "max_depth": [3, 5, 7, 10],
    "subsample": [0.5, 0.7, 1],
    "colsample_bytree": [0.5, 0.7, 1]
}

# Perform Randomized Search
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", eval_metric="mae")
random_search = RandomizedSearchCV(xgb_model, param_grid, cv=5, n_iter=20, scoring="neg_mean_absolute_error", n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best MAE:", -random_search.best_score_)


In [ ]:
import matplotlib.pyplot as plt
import xgboost as xgb

# Extract feature importance
feature_importance = random_search.best_estimator_.feature_importances_
feature_names = X.columns

# Sort features by importance
sorted_idx = feature_importance.argsort()
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align="center")
plt.yticks(range(len(sorted_idx)), [feature_names[i] for i in sorted_idx])
plt.xlabel("Feature Importance")
plt.title("XGBoost Feature Importance")
plt.show()


In [ ]:
import shap

# Fit the SHAP explainer
explainer = shap.Explainer(model, X_test)
shap_values = explainer(X_test)

# Plot feature importance
shap.summary_plot(shap_values, X_test, feature_names=feature_names)


In [ ]:
import seaborn as sns

y_pred_xgb = random_search.best_estimator_.predict(X_test)

plt.figure(figsize=(8, 6))
sns.residplot(x=y_test, y=y_pred_xgb, lowess=True, line_kws={"color": "red"})
plt.xlabel("Actual Values")
plt.ylabel("Residuals (y_actual - y_predicted)")
plt.title("Residual Plot for XGBoost")
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_xgb, alpha=0.6, color="blue")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], "--", color="red")  # 45-degree line
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs. Actual Values")
plt.show()


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred_xgb)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2 = r2_score(y_test, y_pred_xgb)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Predictions
y_pred_svm = grid_search.best_estimator_.predict(X_test)  # SVM
y_pred_xgb = random_search.best_estimator_.predict(X_test)  # XGBoost

# Metrics Calculation
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} Performance:")
    print(f"MAE: {mae}")
    print(f"RMSE: {rmse}")
    print(f"R² Score: {r2}")
    print("-" * 40)
    return mae, rmse, r2

svm_metrics = evaluate_model(y_test, y_pred_svm, "SVM")
xgb_metrics = evaluate_model(y_test, y_pred_xgb, "XGBoost")


In [ ]:
import pandas as pd

# Create DataFrame
comparison_df = pd.DataFrame({
    "Model": ["SVM", "XGBoost"],
    "MAE": [svm_metrics[0], xgb_metrics[0]],
    "RMSE": [svm_metrics[1], xgb_metrics[1]],
    "R² Score": [svm_metrics[2], xgb_metrics[2]]
})

# Display the table
print(comparison_df)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Error calculation
error_svm = y_test - y_pred_svm
error_xgb = y_test - y_pred_xgb

plt.figure(figsize=(10, 5))
sns.histplot(error_svm, bins=30, kde=True, color="blue", label="SVM Errors", alpha=0.6)
sns.histplot(error_xgb, bins=30, kde=True, color="red", label="XGBoost Errors", alpha=0.6)
plt.xlabel("Prediction Error")
plt.ylabel("Frequency")
plt.title("Error Distribution for SVM vs. XGBoost")
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(12, 5))

# SVM plot
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred_svm, alpha=0.6, color="blue")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], "--", color="black")  # 45-degree line
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("SVM: Predicted vs. Actual")

# XGBoost plot
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_xgb, alpha=0.6, color="red")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], "--", color="black")  # 45-degree line
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("XGBoost: Predicted vs. Actual")

plt.tight_layout()
plt.show()


In [ ]:
# Extract feature importance
feature_importance = random_search.best_estimator_.feature_importances_
feature_names = X.columns

# Sort and plot
sorted_idx = feature_importance.argsort()
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align="center")
plt.yticks(range(len(sorted_idx)), [feature_names[i] for i in sorted_idx])
plt.xlabel("Feature Importance")
plt.title("XGBoost Feature Importance")
plt.show()


In [ ]:
import shap

# Fit the SHAP explainer
explainer = shap.Explainer(grid_search.best_estimator_, X_test)
shap_values = explainer(X_test)

# SHAP summary plot
shap.summary_plot(shap_values, X_test, feature_names=feature_names)


In [ ]:
plt.figure(figsize=(12, 5))

# SVM Residual Plot
plt.subplot(1, 2, 1)
sns.residplot(x=y_test, y=y_pred_svm, lowess=True, line_kws={"color": "red"})
plt.xlabel("Actual Values")
plt.ylabel("Residuals")
plt.title("Residual Plot for SVM")

# XGBoost Residual Plot
plt.subplot(1, 2, 2)
sns.residplot(x=y_test, y=y_pred_xgb, lowess=True, line_kws={"color": "red"})
plt.xlabel("Actual Values")
plt.ylabel("Residuals")
plt.title("Residual Plot for XGBoost")

plt.tight_layout()
plt.show()


In [ ]:
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Structure
from matgl.ext.pymatgen import StructureGraphConverter
from matgl.graph.converters import CrystalGraphConverter
import numpy as np
import pandas as pd

API_KEY = "your_api_key_here"
mpr = MPRester(API_KEY)

# Fetching Silicon (Si) crystal structure
structure = mpr.get_structure_by_material_id("mp-149")

# Convert structure to graph representation
graph_converter = CrystalGraphConverter(radius=5.0)
graph = graph_converter.convert(structure)

# Extract features (atomic numbers, bond lengths)
node_features = np.array([site.specie.number for site in structure.sites])
edge_features = np.linalg.norm(graph.edge_attr.numpy(), axis=1)

# Create a DataFrame
df = pd.DataFrame({"Atomic Number": node_features, "Bond Length": edge_features[:len(node_features)]})
df.fillna(df.mean(), inplace=True)  # Handle missing values
print(df.head())


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a network graph
G = nx.Graph()

# Add nodes (atoms) with atomic numbers
for i, atom in enumerate(structure.sites):
    G.add_node(i, element=atom.species_string)

# Add edges (bonds)
for edge in graph.edge_index.T:
    G.add_edge(int(edge[0]), int(edge[1]))

# Plot Graph Representation
plt.figure(figsize=(8, 6))
nx.draw(G, with_labels=True, node_color="lightblue", edge_color="gray", font_weight="bold")
plt.title("Graph Representation of Silicon Crystal")
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Example target variable (Bandgap in eV)
df["Bandgap"] = np.random.uniform(1.0, 3.0, len(df))  # Simulated values

# Splitting data
X = df.drop(columns=["Bandgap"])
y = df["Bandgap"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train models
xgb_model = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1)
xgb_model.fit(X_train, y_train)

svm_model = SVR(kernel="rbf")
svm_model.fit(X_train, y_train)

# Predictions
y_pred_xgb = xgb_model.predict(X_test)
y_pred_svm = svm_model.predict(X_test)


In [ ]:
plt.figure(figsize=(12, 5))

# XGBoost
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred_xgb, color="red", alpha=0.6, label="XGBoost")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], "--", color="black")
plt.xlabel("Actual Bandgap (eV)")
plt.ylabel("Predicted Bandgap (eV)")
plt.title("XGBoost: Predicted vs. Actual Bandgap")
plt.legend()

# SVM
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_svm, color="blue", alpha=0.6, label="SVM")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], "--", color="black")
plt.xlabel("Actual Bandgap (eV)")
plt.ylabel("Predicted Bandgap (eV)")
plt.title("SVM: Predicted vs. Actual Bandgap")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns

error_xgb = y_test - y_pred_xgb
error_svm = y_test - y_pred_svm

plt.figure(figsize=(10, 5))
sns.histplot(error_xgb, bins=20, kde=True, color="red", label="XGBoost Errors", alpha=0.6)
sns.histplot(error_svm, bins=20, kde=True, color="blue", label="SVM Errors", alpha=0.6)
plt.xlabel("Prediction Error")
plt.ylabel("Frequency")
plt.title("Error Distribution for SVM vs. XGBoost")
plt.legend()
plt.show()


In [ ]:
importances = xgb_model.feature_importances_
feature_names = X.columns

sorted_idx = importances.argsort()
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_idx)), importances[sorted_idx], align="center")
plt.yticks(range(len(sorted_idx)), [feature_names[i] for i in sorted_idx])
plt.xlabel("Feature Importance")
plt.title("XGBoost Feature Importance for Semiconductor Properties")
plt.show()


In [ ]:
plt.figure(figsize=(12, 5))

# SVM Residual Plot
plt.subplot(1, 2, 1)
sns.residplot(x=y_test, y=y_pred_svm, lowess=True, line_kws={"color": "red"})
plt.xlabel("Actual Bandgap")
plt.ylabel("Residuals")
plt.title("Residual Plot for SVM")

# XGBoost Residual Plot
plt.subplot(1, 2, 2)
sns.residplot(x=y_test, y=y_pred_xgb, lowess=True, line_kws={"color": "red"})
plt.xlabel("Actual Bandgap")
plt.ylabel("Residuals")
plt.title("Residual Plot for XGBoost")

plt.tight_layout()
plt.show()


In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} Performance:")
    print(f"MAE: {mae}")
    print(f"RMSE: {rmse}")
    print(f"R² Score: {r2}")
    print("-" * 40)
    return mae, rmse, r2

svm_metrics = evaluate_model(y_test, y_pred_svm, "SVM")
xgb_metrics = evaluate_model(y_test, y_pred_xgb, "XGBoost")
